<a href="https://colab.research.google.com/github/drinkersgdl-source/TAREAS-NOE/blob/main/ProyectoModulo1_RodriguezN1_MartinezD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.optimize as opt
import pandas as pd

def get_simple_input(prompt, dtype=float):
    """
    Función auxiliar para obtener entradas. Asume entrada válida (sin try/except).
    """
    if dtype == int:
        return int(input(prompt))
    else:
        return float(input(prompt))

def planeacion_mano_de_obra_novato():
    print("\n--- }Proyecto de Planeación de Mano de Obra (PL) ---")



    N = get_simple_input("1. Ingrese el número de períodos a planificar (ej: 3 semanas): ", dtype=int)

    C_sal = get_simple_input("2. Costo de Salario por trabajador/período ($/trabajador): ")
    C_contrata = get_simple_input("3. Costo de Contratación por trabajador ($/contrato): ")
    C_despido = get_simple_input("4. Costo de Despido por trabajador ($/despido): ")

    R = np.zeros(N)
    print("\n5. Ingrese el número de trabajadores REQUERIDO (R_t):")
    for t in range(N):
        R[t] = get_simple_input(f"   Requisito mínimo Periodo {t+1}: ", dtype=int)

    W_initial = get_simple_input("\n6. Ingrese el número de trabajadores activos INICIALES (W1): ", dtype=int)


    num_W = N
    num_H_L = N - 1

    total_vars = num_W + num_H_L + num_H_L

    c = np.zeros(total_vars)

    c[0 : num_W] = C_sal

    c[num_W : num_W + num_H_L] = C_contrata

    c[num_W + num_H_L : total_vars] = C_despido

    print(f"\nNúmero total de variables de decisión: {total_vars}")


    num_eq_constraints = N
    A_eq = np.zeros((num_eq_constraints, total_vars))
    b_eq = np.zeros(num_eq_constraints)

    A_eq[0, 0] = 1
    b_eq[0] = W_initial


    for t in range(1, N):


        A_eq[t, t-1] = 1
        A_eq[t, t] = -1


        A_eq[t, num_W + (t-1)] = 1
        A_eq[t, num_W + num_H_L + (t-1)] = -1

        b_eq[t] = 0

    num_ub_constraints = N
    A_ub = np.zeros((num_ub_constraints, total_vars))
    b_ub = -R

    for t in range(N):
        A_ub[t, t] = -1

    bounds = (0, None)



    sol = opt.linprog(
        c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        bounds=bounds,
        method='highs'
    )



    print("\n--- Resultados de la Optimización ---")
    print(f"Estado de la Solución: {sol.message}")

    if sol.success:
        costo_minimo = sol.fun
        x_sol = sol.x


        W_sol = x_sol[0 : num_W].round(2)
        H_sol = x_sol[num_W : num_W + num_H_L].round(2)
        L_sol = x_sol[num_W + num_H_L : total_vars].round(2)


        H_full = np.insert(H_sol, 0, 0).round(2)
        L_full = np.insert(L_sol, 0, 0).round(2)

        print(f"\n✅ El **Costo Mínimo Total** es: **${costo_minimo:,.2f}**")
        print("\n--- Distribución Óptima de Mano de Obra por Período ---")


        header = ["Periodo (t)", "W_t (Activos)", "H_t (Contratados)", "L_t (Despedidos)", "Req. Min. (R_t)"]
        periods = np.arange(1, N + 1).reshape(-1, 1)


        results_matrix = np.hstack([
            periods,
            W_sol.reshape(-1, 1),
            H_full.reshape(-1, 1),
            L_full.reshape(-1, 1),
            R.reshape(-1, 1)
        ])

        print(" | ".join(header))
        print("-" * (len(" | ".join(header)) + 2 * N))

        for row in results_matrix:
            print(f" {int(row[0]):<11} | {row[1]:<13} | {row[2]:<17} | {row[3]:<16} | {int(row[4]):<16}")
    else:
        print("\n❌ No se encontró una solución óptima.")
        print("Verifique la coherencia de los datos ingresados.")

if __name__ == "__main__":
    planeacion_mano_de_obra_novato()


--- }Proyecto de Planeación de Mano de Obra (PL) ---
1. Ingrese el número de períodos a planificar (ej: 3 semanas): 4
2. Costo de Salario por trabajador/período ($/trabajador): 2500
3. Costo de Contratación por trabajador ($/contrato): 0
4. Costo de Despido por trabajador ($/despido): 4000

5. Ingrese el número de trabajadores REQUERIDO (R_t):
   Requisito mínimo Periodo 1: 8
   Requisito mínimo Periodo 2: 10
   Requisito mínimo Periodo 3: 12
   Requisito mínimo Periodo 4: 8

6. Ingrese el número de trabajadores activos INICIALES (W1): 8

Número total de variables de decisión: 10

--- Resultados de la Optimización ---
Estado de la Solución: Optimization terminated successfully. (HiGHS Status 7: Optimal)

✅ El **Costo Mínimo Total** es: **$105,000.00**

--- Distribución Óptima de Mano de Obra por Período ---
Periodo (t) | W_t (Activos) | H_t (Contratados) | L_t (Despedidos) | Req. Min. (R_t)
--------------------------------------------------------------------------------------------
 1